In [8]:
import json
import pandas as pd
import os
import re
from lapp.dbms import init_db, inserts, modify, delete, find_by_attr, insert
from lapp.tables import Unit, Vocabulary, GrammarRule, Language, CalligraphyCharacter


In [ ]:
language_id = "de"

## Create a Language

In [ ]:
# Check if the db folder exists, if not create it
if not os.path.exists("../db"):
    os.makedirs("../db")
    print(f"Created directory for database: ../db")

engine, session = init_db()

language = Language(
    id=language_id.upper(),
    name="German",
    native_name="Deutsch",
    level="A1",
    flag="🇩🇪"
)
insert(session, language)

session.close()

2025-07-23 23:27:54,854 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:54,854 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("language")
2025-07-23 23:27:54,855 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,855 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("language")
2025-07-23 23:27:54,856 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,856 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("unit")
2025-07-23 23:27:54,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,857 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("unit")
2025-07-23 23:27:54,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,858 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calligraphy_character")
2025-07-23 23:27:54,858 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,858 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("calligraphy_character")
2025-07-23 23:27:54,858 INFO sqlalchemy.en

## Add Units

In [ ]:
# Initialize the database connection and create db file if it doesn't exist
engine, session = init_db()

# Initialize the database
with open(f'../assets/data/{language_id}/units.json', 'r', encoding="utf8") as f:
    units_array = json.load(f)
    
units = []
for idx, unit_data in enumerate(units_array):
    units.append(
        Unit(
            id = f"{language_id.upper()}_{idx}",
            title = unit_data['title'],
            description = unit_data['description'],
            level = unit_data['level'],
            parent = language,
        )
    )
inserts(session, units)
session.close()

2025-07-23 23:27:54,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:54,894 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("language")
2025-07-23 23:27:54,894 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,894 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("unit")
2025-07-23 23:27:54,897 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,898 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calligraphy_character")
2025-07-23 23:27:54,898 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,899 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("grammar_rule")
2025-07-23 23:27:54,899 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,899 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("vocabulary")
2025-07-23 23:27:54,900 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,900 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("exercises")
2025-07-23 23:27:54,901 INFO sqlalchemy.engi

/Users/noe_jager/Documents/GitHub/vocabulary-dbms/src/lapp/dbms.py:80: SAWarning: Object of type <Unit> not in session, add operation along 'Language.units' will not proceed
  session.commit()


## Add Vocabulary

In [ ]:
# Initialize the database connection and create db file if it doesn't exist
engine, session = init_db()

directory_path = f'../assets/data/{language_id}/vocabulary'

if not os.path.exists(directory_path):
    raise FileNotFoundError(f"No vocabulary found for {language_id}.")

vocs = []

# List all elements (files and directories) in the specified directory and get their full paths
elements_paths = [os.path.join(directory_path, element) for element in os.listdir(directory_path)]
for voc_file in elements_paths:
    if os.path.isfile(voc_file) and voc_file.endswith('.csv'):
        unit_id = re.sub("[^0-9]", "", os.path.basename(voc_file))
        df = pd.read_csv(voc_file)
        for idx, row in df.iterrows():
            vocs.append( 
                Vocabulary(
                    id=f"{language_id.upper()}_{unit_id}_V{idx}",
                    word = row['word'],
                    translation = row["translation"],
                    phonetic = row["pinyin"],
                    example_sentence = row.get("example_sentence", ""),
                    type = row["type"],
                    parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()
                )
            )
inserts(session, vocs)
session.close()

2025-07-23 23:27:54,958 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:54,958 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("language")
2025-07-23 23:27:54,958 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,959 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("unit")
2025-07-23 23:27:54,959 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,959 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calligraphy_character")
2025-07-23 23:27:54,959 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,959 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("grammar_rule")
2025-07-23 23:27:54,960 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,960 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("vocabulary")
2025-07-23 23:27:54,960 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:54,960 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("exercises")
2025-07-23 23:27:54,960 INFO sqlalchemy.engi

/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/2127336807.py:26: SAWarning: Object of type <Vocabulary> not in session, add operation along 'Unit.vocs' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()
/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/2127336807.py:26: SAWarning: Object of type <Vocabulary> not in session, add operation along 'Unit.vocs' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.query(Unit).filter(Unit.id == f"{lan

2025-07-23 23:27:55,107 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit.score AS unit_score, unit.last_seen AS unit_last_seen, unit.language_id AS unit_language_id 
FROM unit 
WHERE unit.id = ?
 LIMIT ? OFFSET ?
2025-07-23 23:27:55,108 INFO sqlalchemy.engine.Engine [cached since 0.145s ago] ('ZH_15', 1, 0)
2025-07-23 23:27:55,109 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit.score AS unit_score, unit.last_seen AS unit_last_seen, unit.language_id AS unit_language_id 
FROM unit 
WHERE unit.id = ?
 LIMIT ? OFFSET ?
2025-07-23 23:27:55,111 INFO sqlalchemy.engine.Engine [cached since 0.1482s ago] ('ZH_15', 1, 0)
2025-07-23 23:27:55,113 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit

/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/2127336807.py:26: SAWarning: Object of type <Vocabulary> not in session, add operation along 'Unit.vocs' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()


2025-07-23 23:27:55,164 INFO sqlalchemy.engine.Engine [cached since 0.2013s ago] ('ZH_14', 1, 0)
2025-07-23 23:27:55,164 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit.score AS unit_score, unit.last_seen AS unit_last_seen, unit.language_id AS unit_language_id 
FROM unit 
WHERE unit.id = ?
 LIMIT ? OFFSET ?
2025-07-23 23:27:55,165 INFO sqlalchemy.engine.Engine [cached since 0.2019s ago] ('ZH_14', 1, 0)
2025-07-23 23:27:55,166 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit.score AS unit_score, unit.last_seen AS unit_last_seen, unit.language_id AS unit_language_id 
FROM unit 
WHERE unit.id = ?
 LIMIT ? OFFSET ?
2025-07-23 23:27:55,166 INFO sqlalchemy.engine.Engine [cached since 0.2033s ago] ('ZH_10', 1, 0)
2025-07-23 23:27:55,166 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_

/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/2127336807.py:26: SAWarning: Object of type <Vocabulary> not in session, add operation along 'Unit.vocs' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()
/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/2127336807.py:26: SAWarning: Object of type <Vocabulary> not in session, add operation along 'Unit.vocs' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.query(Unit).filter(Unit.id == f"{lan

2025-07-23 23:27:55,345 INFO sqlalchemy.engine.Engine [cached since 0.04182s ago] ('ZH_2',)
2025-07-23 23:27:55,345 INFO sqlalchemy.engine.Engine INSERT INTO vocabulary (id, word, translation, phonetic, example_sentence, type, score, last_seen, unit_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-07-23 23:27:55,345 INFO sqlalchemy.engine.Engine [cached since 0.04383s ago] ('ZH_2_V8', '九', 'Neuf', 'Jiǔ', '', 'Chiffres et nombres', 0, '2025-07-23', 'ZH_2')
2025-07-23 23:27:55,345 INFO sqlalchemy.engine.Engine COMMIT
2025-07-23 23:27:55,345 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:55,346 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit.score AS unit_score, unit.last_seen AS unit_last_seen, unit.language_id AS unit_language_id 
FROM unit 
WHERE unit.id = ?
2025-07-23 23:27:55,346 INFO sqlalchemy.engine.Engine [cached since 0.04314s ago] ('ZH_2',)
2025-07-23 23:27:55,346

## Add Grammar Rules

In [ ]:
# Initialize the database connection and create db file if it doesn't exist
engine, session = init_db()

directory_path = f'../assets/data/{language_id}/grammar'

if not os.path.exists(directory_path):
    raise FileNotFoundError(f"No grammar files found for {language_id}.")

grammars = []

# List all elements (files and directories) in the specified directory and get their full paths
elements_paths = [os.path.join(directory_path, element) for element in os.listdir(directory_path)]
for grammar_file in elements_paths:
    if os.path.isfile(grammar_file) and grammar_file.endswith('.json'):
        unit_id = re.sub("[^0-9]", "", os.path.basename(grammar_file))
        with open(grammar_file, 'r', encoding="utf8") as f:
            grammar_data = json.load(f)
        for idx, row in enumerate(grammar_data):
            grammars.append( 
                GrammarRule(
                    id=f"{language_id.upper()}_{unit_id}_G{idx + 1}",
                    title=row['title'],
                    explanation=row["content"],
                    parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()
                )
            )
inserts(session, grammars)
session.close()

2025-07-23 23:27:56,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:56,838 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("language")
2025-07-23 23:27:56,840 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:56,841 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("unit")
2025-07-23 23:27:56,846 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:56,855 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calligraphy_character")
2025-07-23 23:27:56,855 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:56,856 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("grammar_rule")
2025-07-23 23:27:56,856 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:56,857 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("vocabulary")
2025-07-23 23:27:56,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:56,858 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("exercises")
2025-07-23 23:27:56,858 INFO sqlalchemy.engi

/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/1242808380.py:24: SAWarning: Object of type <GrammarRule> not in session, add operation along 'Unit.grammars' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()
/Users/noe_jager/Documents/GitHub/vocabulary-dbms/src/lapp/dbms.py:80: SAWarning: Object of type <GrammarRule> not in session, add operation along 'Unit.grammars' will not proceed
  session.commit()


2025-07-23 23:27:57,069 INFO sqlalchemy.engine.Engine [cached since 0.1098s ago] ('ZH_4_G2', "Le passé d'expérience", "Le passé d'expérience permet de dire ce que l'on a déjà fait ou non. Il nécessite l'emploi du suffixe verbal 过 guo que l'on placera juste après le ve ... (533 characters truncated) ...  chī guo Zhōngguó fàn : « J'ai déjà mangé des plats chinois »\n* 我看过 wǒ kàn guo : « J'ai déjà vu »\n* 我没有看过 wǒ méiyǒu kàn guo : « Je n'ai jamais vu »", 0, '2025-07-23', 'ZH_4')
2025-07-23 23:27:57,069 INFO sqlalchemy.engine.Engine COMMIT
2025-07-23 23:27:57,070 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:57,070 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit.score AS unit_score, unit.last_seen AS unit_last_seen, unit.language_id AS unit_language_id 
FROM unit 
WHERE unit.id = ?
2025-07-23 23:27:57,070 INFO sqlalchemy.engine.Engine [cached since 0.1095s ago] ('ZH_4',

## Add Characters

In [14]:
engine, session = init_db()

directory_path = f'../data/{language_id}/character'

if not os.path.exists(directory_path):
    raise FileNotFoundError(f"No character files found for {language_id}.")

characters = []

# List all elements (files and directories) in the specified directory and get their full paths
elements_paths = [os.path.join(directory_path, element) for element in os.listdir(directory_path)]
for character_file in elements_paths:
    if os.path.isfile(character_file) and character_file.endswith('.csv'):
        unit_id = re.sub("[^0-9]", "", os.path.basename(character_file))
        print(f"Processing: {unit_id}")
        df = pd.read_csv(character_file)
        for idx, row in enumerate(df.itertuples(index=False)):
            characters.append(
                CalligraphyCharacter(
                    id=f"{language_id.upper()}_{unit_id}_C{idx + 1}",
                    character=row.character,
                    components=row.components,
                    phonetic=row.phonetic,
                    meaning=row.meaning,
                    example_word=row.example_word,
                    parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()
                )
            )
inserts(session, characters)
session.close()

2025-07-23 23:27:57,110 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:57,110 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("language")
2025-07-23 23:27:57,110 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:57,111 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("unit")
2025-07-23 23:27:57,111 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:57,111 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calligraphy_character")
2025-07-23 23:27:57,111 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:57,112 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("grammar_rule")
2025-07-23 23:27:57,112 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:57,112 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("vocabulary")
2025-07-23 23:27:57,112 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 23:27:57,112 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("exercises")
2025-07-23 23:27:57,112 INFO sqlalchemy.engi

/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/2084502003.py:26: SAWarning: Object of type <CalligraphyCharacter> not in session, add operation along 'Unit.characters' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.query(Unit).filter(Unit.id == f"{language_id.upper()}_{unit_id}").first()
/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_44096/2084502003.py:26: SAWarning: Object of type <CalligraphyCharacter> not in session, add operation along 'Unit.characters' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  parent=session.quer

2025-07-23 23:27:57,317 INFO sqlalchemy.engine.Engine COMMIT
2025-07-23 23:27:57,319 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 23:27:57,320 INFO sqlalchemy.engine.Engine SELECT unit.id AS unit_id, unit.title AS unit_title, unit.description AS unit_description, unit.level AS unit_level, unit.score AS unit_score, unit.last_seen AS unit_last_seen, unit.language_id AS unit_language_id 
FROM unit 
WHERE unit.id = ?
2025-07-23 23:27:57,320 INFO sqlalchemy.engine.Engine [cached since 0.01628s ago] ('ZH_2',)
2025-07-23 23:27:57,321 INFO sqlalchemy.engine.Engine INSERT INTO calligraphy_character (id, character, components, phonetic, meaning, example_word, score, last_seen, unit_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-07-23 23:27:57,322 INFO sqlalchemy.engine.Engine [cached since 0.01925s ago] ('ZH_2_C6', '四', '-', 'sì', 'Quatre', '十四 shísì : Quatorze', 0, '2025-07-23', 'ZH_2')
2025-07-23 23:27:57,323 INFO sqlalchemy.engine.Engine COMMIT
2025-07-23 23:27:57,324 INFO sqlalchem